In [2]:
import sys

%cd ..
# add the src directory for the code
sys.path.append('src')

%cd .
sys.path.append('cdml-neurips2020/datasets/complexity_10')

/home/nikolas/Downloads/CS-673 (23-24)/Project/msft_causica
/home/nikolas/Downloads/CS-673 (23-24)/Project/msft_causica


In [2]:
# # # pip uninstall torch torchvision torchaudio
# # # pip install torch==1.13.0+cu117 torchvision==0.14.0+cu117 torchaudio==0.13.0+cu117 --extra-index-url https://download.pytorch.org/whl/cu117+
# #  NVIDIA-SMI 536.23                 Driver Version: 536.23       CUDA Version: 12.2
# import torch
# print("CUDA Available: ", torch.cuda.is_available())
# print("CUDA Device Count: ", torch.cuda.device_count())
# if torch.cuda.is_available():
#     print("CUDA Device Name: ", torch.cuda.get_device_name(0))
#     print("CUDA Device Capability: ", torch.cuda.get_device_capability(0))

In [3]:
import os
from operator import itemgetter
import warnings

import fsspec
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch

from pytorch_lightning.callbacks import TQDMProgressBar
from tensordict import TensorDict

import pickle 

from causica.datasets.causica_dataset_format import CAUSICA_DATASETS_PATH, Variable
from causica.distributions import ContinuousNoiseDist
from causica.lightning.data_modules.basic_data_module import BasicDECIDataModule
from causica.lightning.modules.deci_module import DECIModule
from causica.sem.sem_distribution import SEMDistributionModule
from causica.sem.structural_equation_model import ite
from causica.training.auglag import AugLagLRConfig

from tigramite.data_processing import DataFrame
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
from tigramite import plotting as tp
import cdt

warnings.filterwarnings("ignore")
test_run = bool(os.environ.get("TEST_RUN", False))  # used by testing to run the notebook as a script

In [4]:
# Custom Structural Intervention Distance (SID) metric
def compute_sid(predicted_graph, true_graph):
    def get_ancestors(graph):
        ancestors = {}
        for node in graph.nodes():
            ancestors[node] = set(nx.ancestors(graph, node))
        return ancestors

    pred_ancestors = get_ancestors(predicted_graph)
    true_ancestors = get_ancestors(true_graph)

    sid = 0
    for node in true_graph.nodes():
        pred_anc = pred_ancestors.get(node, set())
        true_anc = true_ancestors.get(node, set())
        sid += len(pred_anc.symmetric_difference(true_anc))
    
    return sid

# Function to compute precision, recall, and F1-score for edges
def edge_metrics(true_graph, predicted_graph):
    true_edges = set(true_graph.edges())
    predicted_edges = set(predicted_graph.edges())

    true_positives = len(true_edges & predicted_edges)
    false_positives = len(predicted_edges - true_edges)
    false_negatives = len(true_edges - predicted_edges)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def train_deci(df, variables_spec, constraint_matrix, root_path, name):
    data_module = BasicDECIDataModule(
        dataframe=df,
        variables=[Variable.from_dict(v) for v in variables_spec],
        batch_size=64,
        normalize=True 
    )
    num_nodes = len(data_module.dataset_train.keys())
    outcome = "Y1"
    pl.seed_everything(seed=42)

    lightning_module = DECIModule(
        noise_dist=ContinuousNoiseDist.SPLINE,
        embedding_size=64,
        out_dim_g=3,
        num_layers_g=3,
        num_layers_zeta=3,
        gumbel_temp=10,
        prior_sparsity_lambda=1.0,
        init_rho=1.0,
        init_alpha=0.020,
        auglag_config=AugLagLRConfig(
            max_inner_steps=1500,
            max_outer_steps=8,
            lr_init_dict={
                "icgnn": 0.00076,
                "vardist": 0.0098,
                "functional_relationships": 3e-3,
                "noise_dist": 0.0070,
            },
        ),
    )

    lightning_module.constraint_matrix = torch.tensor(constraint_matrix)

    trainer = pl.Trainer(
        accelerator="auto",
        max_epochs=3000,
        fast_dev_run=False,
        callbacks=[TQDMProgressBar(refresh_rate=19)],
        enable_checkpointing=False
    )

    trainer.fit(lightning_module, data_module)
    
    trained_name = name + 'deci_learned_SEM.pt'
    save_model_path = os.path.join(root_path, trained_name)
    torch.save(lightning_module.sem_module, save_model_path)
    return lightning_module

def run_pcmci(df):
    data_array = df.values
    tigramite_df = DataFrame(data_array, var_names=df.columns)
    parcorr = ParCorr()
    pcmci = PCMCI(dataframe=tigramite_df, cond_ind_test=parcorr, verbosity=1)
    max_lag = max([int(node.split(":")[1]) for node in df.columns if ":" in node])
    results = pcmci.run_pcmci(tau_max=max_lag, pc_alpha=None)
    link_matrix = results['graph']

    pcmci_graph = nx.DiGraph()
    for i in range(link_matrix.shape[0]):
        for j in range(link_matrix.shape[1]):
            if link_matrix[i, j] != 0:
                pcmci_graph.add_edge(df.columns[i], df.columns[j])
    return pcmci_graph

def evaluate_model(graph, gt_graph):
    pcmci_adj_matrix = nx.to_numpy_array(graph, weight=None).astype(np.float64)
    gt_adj_matrix = nx.to_numpy_array(gt_graph, weight=None).astype(np.float64)
    pcmci_graph_float = nx.from_numpy_array(pcmci_adj_matrix, create_using=nx.DiGraph)
    gt_graph_float = nx.from_numpy_array(gt_adj_matrix, create_using=nx.DiGraph)
    shd = cdt.metrics.SHD(gt_graph_float, pcmci_graph_float, double_for_anticausal=True)
    aupr, curve = cdt.metrics.precision_recall(gt_graph_float, pcmci_graph_float, low_confidence_undirected=True)
    sid = compute_sid(graph, gt_graph)
    precision, recall, f1_score = edge_metrics(gt_graph, graph)
    return shd, aupr, sid, precision, recall, f1_score

root_path = '../cdml-neurips2020/datasets/'
complexities = ['complexity_0']
# complexities = ['complexity_0', 'complexity_10', 'complexity_30']

# File paths for DECI and PCMCI evaluation metrics
deci_metrics_file = os.path.join(root_path, 'deci_evaluation_metrics.csv')
pcmci_metrics_file = os.path.join(root_path, 'pcmci_evaluation_metrics.csv')

# Initialize metrics files
with open(deci_metrics_file, 'w') as f:
    f.write('Dataset,SHD,AUPR,SID,Precision,Recall,F1-Score\n')
with open(pcmci_metrics_file, 'w') as f:
    f.write('Dataset,SHD,AUPR,SID,Precision,Recall,F1-Score\n')

for complexity in complexities:
    dataset_path = os.path.join(root_path, complexity)
    for filename in os.listdir(dataset_path):
        if filename.endswith('-lagged.csv'):
            df = pd.read_csv(os.path.join(dataset_path, filename))
            name = filename.replace('-lagged.csv', '')
            variables_spec = []
            for col in df.columns:
                variables_spec.append({"name": col, "type": "continuous", "group_name": col})
            
            variables_path = os.path.join(dataset_path, filename.replace('.csv', '.json'))
            with fsspec.open(variables_path, mode="w", encoding="utf-8") as f:
                json.dump({"variables": variables_spec}, f, indent=2)

            with fsspec.open(variables_path, mode="r", encoding="utf-8") as f:
                variables_spec = json.load(f)["variables"]
            
            # Generate constraint matrix
            node_names = df.columns.tolist()
            num_nodes = len(node_names)
            max_lag = max([int(node.split(":")[1]) for node in node_names if ":" in node])
            constraint_matrix = np.full((num_nodes, num_nodes), np.nan, dtype=np.float32)
            node_name_to_idx = {key: i for i, key in enumerate(node_names)}
            
            for node in node_names:
                for node_2 in node_names:
                    suffix = node.split(":")[1] if ":" in node else None
                    suffix_2 = node_2.split(":")[1] if ":" in node_2 else None
                    if suffix is not None and suffix_2 is not None and suffix == suffix_2:
                        constraint_matrix[node_name_to_idx[node], node_name_to_idx[node_2]] = 0
                        constraint_matrix[node_name_to_idx[node_2], node_name_to_idx[node]] = 0
                    if ":" not in node:
                        constraint_matrix[node_name_to_idx[node], :] = 0
                    prefix_2 = node_2.split(":")[0]
                    if node == node_2:
                        constraint_matrix[node_name_to_idx[node], node_name_to_idx[node_2]] = 0
                    if prefix_2 == node:
                        constraint_matrix[node_name_to_idx[node], node_name_to_idx[node_2]] = 0
                    if ":" in node_2 and node == node_2.split(":")[1]:
                        constraint_matrix[node_name_to_idx[node], node_name_to_idx[node_2]] = 0
                        constraint_matrix[node_name_to_idx[node_2], node_name_to_idx[node]] = 0
                    if ":" in node and ":" in node_2 and int(node.split(":")[1]) < int(node_2.split(":")[1]):
                        constraint_matrix[node_name_to_idx[node], node_name_to_idx[node_2]] = 0
                    if ":" not in node and ":" in node_2:
                        constraint_matrix[node_name_to_idx[node], node_name_to_idx[node_2]] = 0            

            # Train DECI
            lightning_module = train_deci(df, variables_spec, constraint_matrix, dataset_path, name)

            # Run PCMCI
            pcmci_graph = run_pcmci(df)

            # Load and clean ground truth graph
            graph_filename = os.path.join(dataset_path, filename.split('-')[0] + '-causal_graph.pkl')
            with open(graph_filename, 'rb') as f:
                gt_graph = pickle.load(f)

            for node in list(gt_graph.nodes):
                if node.startswith('S'):
                    gt_graph.remove_node(node)
            for edge in list(gt_graph.edges):
                if edge[0].startswith('S') or edge[1].startswith('S'):
                    gt_graph.remove_edge(edge[0], edge[1])
            for node in list(gt_graph.nodes):
                new_name = node.replace('_t', '').replace('_', '_').replace(':1', ':1').replace('-', ':')
                nx.relabel_nodes(gt_graph, {node: new_name}, copy=False)

            # Evaluate DECI model
            deci_graph = nx.from_numpy_array(lightning_module.sem_module().mode.graph.cpu().numpy(), create_using=nx.DiGraph)
            deci_graph = nx.relabel_nodes(deci_graph, {i: key for i, key in enumerate(df.columns)})
            deci_shd, deci_aupr, deci_sid, deci_precision, deci_recall, deci_f1_score = evaluate_model(deci_graph, gt_graph)
            print(f'DECI - Dataset: {name}, SHD: {deci_shd}, AUPR: {deci_aupr}, SID: {deci_sid}, Precision: {deci_precision}, Recall: {deci_recall}, F1-Score: {deci_f1_score}')
            with open(deci_metrics_file, 'a') as f:
                f.write(f'{name},{deci_shd},{deci_aupr},{deci_sid},{deci_precision},{deci_recall},{deci_f1_score}\n')

            # Evaluate PCMCI model
            pcmci_shd, pcmci_aupr, pcmci_sid, pcmci_precision, pcmci_recall, pcmci_f1_score = evaluate_model(pcmci_graph, gt_graph)
            print(f'PCMCI - Dataset: {name}, SHD: {pcmci_shd}, AUPR: {pcmci_aupr}, SID: {pcmci_sid}, Precision: {pcmci_precision}, Recall: {pcmci_recall}, F1-Score: {pcmci_f1_score}')
            with open(pcmci_metrics_file, 'a') as f:
                f.write(f'{name},{pcmci_shd},{pcmci_aupr},{pcmci_sid},{pcmci_precision},{pcmci_recall},{pcmci_f1_score}\n')

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type                  | Params
------------------------------------------------------
0 | auglag_loss | AugLagLossCalculator  | 0     
1 | sem_module  | SEMDistributionModule | 28.1 K
------------------------------------------------------
28.0 K    Trainable params
64        Non-trainable params
28.1 K    Total params
0.112     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Updating alpha to: 0.09999999403953552


`Trainer.fit` stopped: `max_epochs=3000` reached.


Updating alpha to: 0.4999999701976776

##
## Step 1: PC1 algorithm for selecting lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 1
pc_alpha = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
max_conds_dim = None
max_combinations = 1




...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type                  | Params
------------------------------------------------------
0 | auglag_loss | AugLagLossCalculator  | 0     
1 | sem_module  | SEMDistributionModule | 60.3 K
------------------------------------------------------
59.7 K    Trainable params
576       Non-trainable params
60.3 K    Total params
0.241     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

Updating alpha to: 0.09999999403953552
Updating alpha to: 0.4999999701976776
Updating alpha to: 2.499999761581421
Updating alpha to: 12.499999046325684
Updating alpha to: 62.499996185302734

##
## Step 1: PC1 algorithm for selecting lagged conditions
##

Parameters:
independence test = par_corr
tau_min = 1
tau_max = 3
pc_alpha = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
max_conds_dim = None
max_combinations = 1




: 

: 